In [35]:
# This code is a dud, it doesn't work because even llama3 
# isn't compatible with openai prompt template, however I 
# might be able to fix it by making a prompt by myself, I 
# will look back at it again

In [19]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [20]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)
wiki_tool = WikipediaQueryRun(api_wrapper=api_wrapper)

In [21]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents,OllamaEmbeddings())
retriever=vectordb.as_retriever()

In [22]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,"langsmith_search",
                      "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!")

In [23]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper=ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)

In [24]:
print(wiki_tool.name)
print(retriever_tool.name)
print(arxiv.name)

wikipedia
langsmith_search
arxiv


In [25]:
tools = [wiki_tool, retriever_tool, arxiv]

In [26]:
from langchain_community.chat_models import ChatOllama
## Load Ollama LAMA2 LLM model
llm=ChatOllama(model="llama3")

In [27]:
from langchain import hub
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

c:\Users\JSK\AppData\Local\Programs\Python\Python39\lib\site-packages\langsmith\client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [28]:
### Agents
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,prompt)

In [ ]:
## Agent Executer
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [34]:
agent_executor.invoke({"input":"Tell me about Langsmith"})



> Entering new AgentExecutor chain...
Langsmith!

Langsmith is an ancient settlement located in the state of Kansas, United States. The site was once inhabited by the Kansa tribe and later became a small town during the 19th century.

Here are some interesting facts about Langsmith:

1. **Kansa Heritage**: As mentioned earlier, Langsmith was initially settled by the Kansa tribe, who were an important part of the region's Native American history.
2. **Pioneer Era**: In the early 19th century, European-American settlers began to arrive at Langsmith, attracted by its fertile land and access to the Kansas River. The area became a small farming community.
3. **Town Establishment**: Langsmith was officially founded in 1857, with a post office established shortly after. The town grew slowly but steadily, with various businesses and services emerging over time.
4. **Railroad Connection**: In 1869, the Leavenworth, Lawrence and Denver City Railroad (later part of the Atchison, Topeka and Sant

{'input': 'Tell me about Langsmith',
 'output': "Langsmith!\n\nLangsmith is an ancient settlement located in the state of Kansas, United States. The site was once inhabited by the Kansa tribe and later became a small town during the 19th century.\n\nHere are some interesting facts about Langsmith:\n\n1. **Kansa Heritage**: As mentioned earlier, Langsmith was initially settled by the Kansa tribe, who were an important part of the region's Native American history.\n2. **Pioneer Era**: In the early 19th century, European-American settlers began to arrive at Langsmith, attracted by its fertile land and access to the Kansas River. The area became a small farming community.\n3. **Town Establishment**: Langsmith was officially founded in 1857, with a post office established shortly after. The town grew slowly but steadily, with various businesses and services emerging over time.\n4. **Railroad Connection**: In 1869, the Leavenworth, Lawrence and Denver City Railroad (later part of the Atchiso

In [33]:
agent_executor.invoke({"input":"What's the paper 1605.08386 about?"})



> Entering new AgentExecutor chain...
The paper "1605.08386" seems to be a research article in the field of physics, specifically in the area of cosmology.

According to my analysis, this paper titled "Observational evidence for anisotropies in the cosmic microwave background" was published on April 22, 2006, and it reports on observations made by the Wilkinson Microwave Anisotropy Probe (WMAP) satellite. The WMAP mission was designed to map the cosmic microwave background radiation (CMB), which is thought to be a remnant of the Big Bang.

The authors of this paper analyzed the data collected by WMAP and found strong evidence for anisotropies, or patterns, in the CMB. These anisotropies are tiny fluctuations in the temperature of the CMB that can be used to learn about the properties of the universe when it was just 380,000 years old.

The authors concluded that these anisotropies provide a robust test of models for the early universe and support the Big Bang theory. The results have

{'input': "What's the paper 1605.08386 about?",
 'output': 'The paper "1605.08386" seems to be a research article in the field of physics, specifically in the area of cosmology.\n\nAccording to my analysis, this paper titled "Observational evidence for anisotropies in the cosmic microwave background" was published on April 22, 2006, and it reports on observations made by the Wilkinson Microwave Anisotropy Probe (WMAP) satellite. The WMAP mission was designed to map the cosmic microwave background radiation (CMB), which is thought to be a remnant of the Big Bang.\n\nThe authors of this paper analyzed the data collected by WMAP and found strong evidence for anisotropies, or patterns, in the CMB. These anisotropies are tiny fluctuations in the temperature of the CMB that can be used to learn about the properties of the universe when it was just 380,000 years old.\n\nThe authors concluded that these anisotropies provide a robust test of models for the early universe and support the Big Ban